## Notebook 02: Fine-Tuning con Unsloth

Este notebook implementa el fine-tuning de un modelo Llama-3 usando LoRA (Low-Rank Adaptation) para crear el estilo de Alicia en el País de las Maravillas.

### 1. Instalación de herramientas 

**Nota:** Esta celda está diseñada para ejecutarse en **Google Colab**. Si trabajas en **local**, asegúrate de tener estas librerías ya instaladas en tu entorno Python.

⚠️ **Requisito:** Se necesita una **GPU con CUDA** para ejecutar este notebook. En Colab, usa un runtime con GPU (Runtime > Change runtime type > T4 GPU).

In [ ]:
!pip install unsloth xformers trl peft accelerate bitsandbytes

### 2. Importar librerías

In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

### 3. Cargar modelo Llama-3 (optimizado para 4-bit)

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth\Qwen3-0.6B-Base-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)

### 3.5. Prueba del modelo base (antes del fine-tuning)

Vamos a probar el modelo base con algunas frases para ver cómo responde antes del entrenamiento.

In [ ]:
# Frases de prueba
test_phrases = [
    "Era una tarde soleada y los niños jugaban en el parque",
    "El gato dormía plácidamente en el sofá",
    "María caminaba por la calle cuando empezó a llover"
]

# Preparar el modelo para inferencia
FastLanguageModel.for_inference(model)

# Formato del prompt (el mismo que usaremos en el entrenamiento)
prompt_template = """A continuación se muestra una instrucción que describe una tarea.
Escribe una respuesta que complete adecuadamente la petición.

### Instrucción:
Reescribe el siguiente texto con el estilo de Alicia en el País de las Maravillas.

### Entrada:
{}

### Respuesta:
"""

print("=" * 80)
print("RESULTADOS DEL MODELO BASE (SIN FINE-TUNING)")
print("=" * 80)

for i, phrase in enumerate(test_phrases, 1):
    print(f"\n📝 Frase {i}: {phrase}")
    print("-" * 80)
    
    # Crear el prompt
    prompt = prompt_template.format(phrase)
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Generar respuesta
    outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.7, top_p=0.9)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraer solo la respuesta (después de "### Respuesta:")
    if "### Respuesta:" in response:
        response = response.split("### Respuesta:")[-1].strip()
    
    print(f"🤖 Respuesta: {response}")
    print()

print("=" * 80)

### 4. Configurar LoRA

Solo entrenamos el 1-2% de los parámetros usando LoRA (Low-Rank Adaptation).

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
)

### 5. Formatear el Dataset

Formato Instrucción-Input-Respuesta (formato Alpaca) para el prompt engineering.

In [ ]:
prompt_format = """A continuación se muestra una instrucción que describe una tarea.
Escribe una respuesta que complete adecuadamente la petición.

### Instrucción:
{}

### Entrada:
{}

### Respuesta:
{}"""

def formatting_prompts_func(examples):
    # The instruction is constant for all examples in the batch.
    constant_instruction = "Reescribe el siguiente texto con el estilo de Alicia en el País de las Maravillas."
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    # Iterate over inputs and outputs, applying the constant instruction to each pair.
    for input_text, output_text in zip(inputs, outputs):
        text = prompt_format.format(constant_instruction, input_text, output_text)
        texts.append(text)
    return { "text" : texts, }

dataset = load_dataset("json", data_files="/content/synthetic_alicia.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

### 6. Entrenamiento

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    tokenizer = tokenizer, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "outputs",
    ),
)
trainer.train()

### 6.5. Prueba del modelo fine-tuneado

Ahora vamos a probar el modelo entrenado con las mismas frases para comparar.

In [ ]:
# Preparar el modelo entrenado para inferencia
FastLanguageModel.for_inference(model)

print("=" * 80)
print("RESULTADOS DEL MODELO FINE-TUNEADO")
print("=" * 80)

for i, phrase in enumerate(test_phrases, 1):
    print(f"\n📝 Frase {i}: {phrase}")
    print("-" * 80)
    
    # Crear el prompt
    prompt = prompt_template.format(phrase)
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    
    # Generar respuesta
    outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.7, top_p=0.9)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extraer solo la respuesta (después de "### Respuesta:")
    if "### Respuesta:" in response:
        response = response.split("### Respuesta:")[-1].strip()
    
    print(f"✨ Respuesta: {response}")
    print()

print("=" * 80)

### 7. Guardar el modelo

Tenemos dos opciones: guardar solo los adaptadores LoRA (rápido) o exportar a GGUF (lento pero necesario para Ollama).

#### 7.1. Guardar adaptadores LoRA (RÁPIDO - Recomendado primero)

Esta opción es **muy rápida** (segundos) y guarda solo los pesos entrenados (adaptadores LoRA). El archivo resultante pesa solo unos pocos MB.

In [ ]:
# Guardar solo los adaptadores LoRA (muy rápido)
model.save_pretrained("lora_model_alicia")
tokenizer.save_pretrained("lora_model_alicia")

print("✅ Adaptadores LoRA guardados en 'lora_model_alicia/'")
print("📦 Tamaño: ~50-100 MB")
print("⏱️ Tiempo: unos segundos")

#### 7.2. Exportar a GGUF para Ollama (LENTO - Solo si vas a usar Ollama)

⚠️ **ADVERTENCIA**: Este proceso puede tardar **10-30 minutos** o más. Solo ejecuta esta celda si necesitas usar el modelo en Ollama.

In [ ]:
# Este proceso puede tardar 10-30 minutos
print("⏳ Iniciando exportación a GGUF...")
print("⚠️ Este proceso puede tardar entre 10-30 minutos")

model.save_pretrained_gguf("model_alicia", tokenizer, quantization_method = "q4_k_m")

print("✅ Modelo exportado a GGUF en 'model_alicia/'")


### 8. Cómo cargar el modelo guardado

Dependiendo de cómo guardaste el modelo, tienes diferentes opciones.

#### 8.1. Opción A: Cargar adaptadores LoRA en Python (si guardaste con 7.1)

Puedes cargar los adaptadores LoRA directamente en Python para hacer inferencia.

In [ ]:
# Ejemplo de cómo cargar el modelo con adaptadores LoRA guardados
from unsloth import FastLanguageModel

# Cargar el modelo base + adaptadores LoRA
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model_alicia",  # Carpeta donde guardamos
    max_seq_length = 2048,
    load_in_4bit = True,
)

# Preparar para inferencia
FastLanguageModel.for_inference(model)

# Ya puedes usar el modelo
prompt = """A continuación se muestra una instrucción que describe una tarea.
Escribe una respuesta que complete adecuadamente la petición.

### Instrucción:
Reescribe el siguiente texto con el estilo de Alicia en el País de las Maravillas.

### Entrada:
Era una tarde soleada

### Respuesta:
"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#### 8.2. Opción B: Usar con Ollama (si exportaste a GGUF en 7.2)

Una vez exportado el modelo a GGUF, necesitas cargarlo en Ollama para usarlo.

##### Paso 1: Crear un Modelfile

Crea un archivo llamado `Modelfile` en tu directorio local con el siguiente contenido:

In [ ]:
%%writefile Modelfile
FROM ./model_alicia-Q4_K_M.gguf

TEMPLATE """A continuación se muestra una instrucción que describe una tarea.
Escribe una respuesta que complete adecuadamente la petición.

### Instrucción:
Reescribe el siguiente texto con el estilo de Alicia en el País de las Maravillas.

### Entrada:
{{ .Prompt }}

### Respuesta:
"""

PARAMETER temperature 0.7
PARAMETER top_p 0.9
PARAMETER stop "###"

##### Paso 2: Importar el modelo a Ollama

Ejecuta este comando en tu terminal local (no en Colab):

In [ ]:
# En terminal local:
# ollama create alicia -f Modelfile

##### Paso 3: Usar el modelo

Desde terminal:

In [ ]:
# En terminal:
# ollama run alicia "Era una tarde soleada y los niños jugaban en el parque"

O desde Python con la API de Ollama:

In [ ]:
import requests

response = requests.post('http://localhost:11434/api/generate', 
    json={
        'model': 'alicia',
        'prompt': 'Era una tarde soleada y los niños jugaban en el parque',
        'stream': False
    }
)

print(response.json()['response'])